In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
# Core logic specific to project/report

import src.config
import pandas as pd
from deltalake import DeltaTable
from pathlib import Path
import cdutils.deduplication
import src.loan_trial.fetch_data

# def main_pipeline():
# Main loan data, silver table
TABLE_PATH = src.config.SILVER / "account"
accts = DeltaTable(TABLE_PATH).to_pandas()

MACRO_TYPE_MAPPING = {
        'CML': 'Loan',
        'MLN': 'Loan',
        'CNS': 'Loan',
        'MTG': 'Loan',
        'CK': 'Deposit',
        'SAV': 'Deposit',
        'TD': 'Deposit'
    }

accts['Macro Account Type'] = accts['mjaccttypcd'].map(MACRO_TYPE_MAPPING)
accts = accts[accts['Macro Account Type'] == 'Loan'].copy()

accts = accts[[
    'acctnbr',
    'ownersortname',
    'mjaccttypcd',
    'currmiaccttypcd',
    'product',
    'curracctstatcd',
    'noteintrate',
    'notebal',
    'bookbalance',
    'branchname',
    'loanofficer',
    'contractdate',
    'datemat',
    'creditlimitamt',
    'loanlimityn',
]].copy()

# Get investor data
invr = src.loan_trial.fetch_data.fetch_invr()
wh_invr = invr['wh_invr'].copy()


acctgrpinvr = invr['acctgrpinvr'].copy()

wh_org = DeltaTable(src.config.BRONZE / "wh_org").to_pandas()
wh_org = wh_org[[
    'orgnbr',
    'orgname'
]].copy()
dedupe_list = [
    {'df':wh_org, 'field':'orgnbr'}
]
wh_org = cdutils.deduplication.dedupe(dedupe_list).copy()
wh_org['orgnbr'] = wh_org['orgnbr'].astype(str)
wh_invr['acctgrpnbr'] = wh_invr['acctgrpnbr'].astype(str)
acctgrpinvr['acctgrpnbr'] = wh_invr['acctgrpnbr'].astype(str)
acctgrpinvr['invrorgnbr'] = acctgrpinvr['invrorgnbr'].astype(str)

merged_investor = wh_invr.merge(acctgrpinvr, on='acctgrpnbr', how='left').merge(wh_org, left_on='invrorgnbr', right_on='orgnbr')
merged_investor = merged_investor.sort_values(by='pctowned', ascending=False).copy()
dedupe_list = [
    {'df':merged_investor, 'field':'acctnbr'}
]
merged_investor = cdutils.deduplication.dedupe(dedupe_list)
merged_investor = merged_investor.drop(columns=['orgnbr','invrorgnbr','pctowned','acctgrpnbr']).copy()
merged_investor['acctnbr'] = merged_investor['acctnbr'].astype(str)
assert merged_investor['acctnbr'].is_unique, "Duplicates exist. Pre-merge of investor data to full df"

merged_investor = merged_investor.rename(columns={
    'orgname':'Investor Name',
    'originvrrate':'Orig Investor Rate',
    'currinvrrate':'Current Investor Rate'
}).copy()

accts = accts.merge(merged_investor, on='acctnbr', how='left')
# return accts


# notenextratechange (WH_ACCTCOMMON)
# noteratechangecalpercd (WH_ACCTCOMMON)
# rcf
#H



In [ ]:
accts

In [ ]:
merged_investor

In [ ]:
accts

In [ ]:
wh_invr

In [ ]:
acctgrpinvr

In [ ]:
merged_investor

In [ ]:
import cdutils.database.connect # type: ignore
from sqlalchemy import text # type: ignore
from datetime import datetime
from typing import Optional

def fetch_rtxnbal():
    participant = text("""
    SELECT
        a.*
    FROM
        OSIBANK.RPT_MM_GLINT a
    """)

    queries = [
        # {
        #     'key': 'wh_rtxn',
        #     'sql': wh_rtxn.bindparams(
        #         start_date=start_date.strftime('%Y-%m-%d'),
        #         end_date=end_date.strftime('%Y-%m-%d')
        #     ),
        #     'engine': 2
        # },
        {
            'key': 'participant',
            'sql': participant,
            'engine': 1
        },
    ]

    data = cdutils.database.connect.retrieve_data(queries)
    return data




In [ ]:
data = fetch_rtxnbal()

In [ ]:
participant = data['participant'].copy()

In [ ]:
participant